In [408]:
import pandas as pd
import vectorbt as vbt
import pandas_ta as ta
import numpy as np
import yaml

with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

df_day = pd.read_csv(config['Data_filename_day'], parse_dates=['Time'], index_col='Time')
df_day.drop(columns=['Volume'], inplace=True)
# df_day['EMA10'] = ta.ema(df_day['Close'])
# df_day['EMA50'] = ta.ema(df_day['Close'], length=50)
# df_day.dropna(inplace=True)
# df_day['MTI'] = (df_day['EMA10'] - df_day['EMA50'])
# mti_green_mask = ((df_day['MTI'] > 0.1) & (df_day['Close'].shift(1) > df_day['Close'].shift(2)))
# df_day.loc[(df_day['MTI'] > 0.1),'COLOR'] = 'G'
# df_day.loc[df_day['MTI'] < -0.1,'COLOR'] = 'R'
# df_day.loc[(df_day['MTI'] < 0.1) & (df_day['MTI'] > -0.1),'COLOR'] = 'Y'
df_day['SMA20'] = ta.sma(df_day['Close'], length=20)
df_day['SMA50'] = ta.sma(df_day['Close'], length=50)
df_day.dropna(inplace=True)
df_day['Color'] = 'Y'
df_day.loc[df_day['SMA20'] < df_day['SMA50'],'Color'] = 'R'
df_day.loc[(df_day['SMA20'] > df_day['SMA50']) & (df_day['Close'] > df_day['SMA20']) & (df_day['SMA20'] > df_day['SMA20'].shift(7)),'Color'] = 'G'
df_day.drop(columns=['SMA20', 'SMA50'], inplace=True)
# df_day.tail(50)

In [409]:
def chandelier_exit(df_day, length=22, mult=3.0, use_close=True):
        
    df = df_day.copy()
    
    # Calculate ATR
    high = df['High']
    low = df['Low']
    close = df['Close']
    
    # True Range calculation
    tr1 = high - low
    tr2 = abs(high - close.shift(1))
    tr3 = abs(low - close.shift(1))
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    
    # ATR using Wilder's smoothing (EMA with alpha = 1/length)
    atr = tr.ewm(alpha=1/length, adjust=False).mean()
    atr_mult = mult * atr
    
    # Calculate long stop
    if use_close:
        highest = close.rolling(window=length).max()
    else:
        highest = high.rolling(window=length).max()
    
    long_stop = highest - atr_mult
    
    # Apply long stop logic: can only move up or stay same
    long_stop_prev = long_stop.shift(1)
    close_prev = close.shift(1)
    
    for i in range(1, len(long_stop)):
        if pd.notna(long_stop_prev.iloc[i]) and pd.notna(close_prev.iloc[i]):
            if close_prev.iloc[i] > long_stop_prev.iloc[i]:
                long_stop.iloc[i] = max(long_stop.iloc[i], long_stop_prev.iloc[i])
    
    # Calculate short stop
    # if use_close:
    #     lowest = close.rolling(window=length).min()
    # else:
    #     lowest = low.rolling(window=length).min()
    
    # short_stop = lowest + atr_mult
    
    # # Apply short stop logic: can only move down or stay same
    # short_stop_prev = short_stop.shift(1)
    
    # for i in range(1, len(short_stop)):
    #     if pd.notna(short_stop_prev.iloc[i]) and pd.notna(close_prev.iloc[i]):
    #         if close_prev.iloc[i] < short_stop_prev.iloc[i]:
    #             short_stop.iloc[i] = min(short_stop.iloc[i], short_stop_prev.iloc[i])
    
    # Determine direction
    # direction = pd.Series(index=df.index, dtype=int)
    # direction.iloc[0] = 1  # Start with long
    
    # for i in range(1, len(direction)):
    #     if pd.notna(short_stop_prev.iloc[i]) and close.iloc[i] > short_stop_prev.iloc[i]:
    #         direction.iloc[i] = 1
    #     elif pd.notna(long_stop_prev.iloc[i]) and close.iloc[i] < long_stop_prev.iloc[i]:
    #         direction.iloc[i] = -1
    #     else:
    #         direction.iloc[i] = direction.iloc[i-1]
    
    # Generate signals
    # direction_prev = direction.shift(1)
    # buy_signal = (direction == 1) & (direction_prev == -1)
    # sell_signal = (direction == -1) & (direction_prev == 1)
    
    # Add indicator columns to the original dataframe
    df['Ch_exit'] = long_stop
    # df['short_stop'] = short_stop
    # df['direction'] = direction
    # df['buy_signal'] = buy_signal
    # df['sell_signal'] = sell_signal
    # df['atr'] = atr
    
    return df

df_day = chandelier_exit(df_day, use_close=False)
df_day.dropna(inplace=True)
# df_day.tail(50)

In [410]:
index_arr_day = df_day.index.to_numpy()
close_arr_day = df_day['Close'].to_numpy()
color_arr_day = df_day['Color'].to_numpy()
ch_exit_arr_day = df_day['Ch_exit'].to_numpy()
entry_arr_day = np.full(len(index_arr_day), False)
exit_arr_day = np.full(len(index_arr_day), False)
price_arr_day = np.full(len(index_arr_day), np.nan)

def backtest(df_day):

    trade_is_open = False
    close_price = np.nan

    for i in range(1, len(index_arr_day)):
        if not trade_is_open:
            if color_arr_day[i] == 'G':
                trade_is_open = True
                entry_arr_day[i] = True
                close_price = ch_exit_arr_day[i]
                price_arr_day[i] = close_arr_day[i]
            else:
                continue
        elif trade_is_open:
            if (close_arr_day[i] > close_price):
                stop = i + config['Days'] + 1
                if (stop >= len(index_arr_day)):
                    continue
                go_throw = True
                for j in range(i, stop):
                    if (j >= len(index_arr_day)):
                        break
                    if (close_arr_day[j] < close_price):
                        go_throw = False
                        break
                if go_throw:
                    trade_is_open = False
                    exit_arr_day[j] = True
                    close_price = None
                    price_arr_day[j] = close_arr_day[j]
                # j = i + config['Days']
                # if (j >= len(index_arr_day)):
                #     continue
                # trade_is_open = False
                # exit_arr_day[j] = True
                # close_price = None
                # price_arr_day[j] = close_arr_day[j]
            else:
                continue

    return df_day

df_day = backtest(df_day)

In [411]:
pf = vbt.Portfolio.from_signals(
entries = entry_arr_day,
exits = exit_arr_day,
price = price_arr_day,
open = df_day["Open"],
close = close_arr_day,
size = config['Trade']['size'],
size_type = config['Trade']['size_type'],
init_cash = config['Initial_cash'],
freq = '1d'
)

In [412]:
pf.stats()

Start                          2014-07-08 13:30:00+00:00
End                            2025-12-15 14:30:00+00:00
Period                               64808 days 00:00:00
Start Value                                      10000.0
End Value                                    9927.078845
Total Return [%]                               -0.729212
Benchmark Return [%]                          986.186924
Max Gross Exposure [%]                           3.25587
Total Fees Paid                                      0.0
Max Drawdown [%]                                 2.82545
Max Drawdown Duration                61942 days 00:00:00
Total Trades                                          91
Total Closed Trades                                   91
Total Open Trades                                      0
Open Trade PnL                                       0.0
Win Rate [%]                                   42.857143
Best Trade [%]                                 25.489489
Worst Trade [%]                

In [413]:
pf.trades.records_readable.tail(50)

,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
41,41,Open,1.0,2020-07-29 13:30:00+00:00,76.1010,0.0,2020-07-30 13:30:00+00:00,76.5725,0.0,0.4715,0.006196,Long,Closed,41
42,42,Open,1.0,2020-08-21 13:30:00+00:00,79.0210,0.0,2020-09-04 13:30:00+00:00,79.5520,0.0,0.5310,0.006720,Long,Closed,42
43,43,Open,1.0,2020-11-03 14:30:00+00:00,82.5105,0.0,2020-12-10 14:30:00+00:00,88.7665,0.0,6.2560,0.075821,Long,Closed,43
44,44,Open,1.0,2021-01-22 14:30:00+00:00,95.0525,0.0,2021-02-26 14:30:00+00:00,101.8430,0.0,6.7905,0.071439,Long,Closed,44
45,45,Open,1.0,2021-03-01 14:30:00+00:00,104.0755,0.0,2021-03-02 14:30:00+00:00,103.7920,0.0,-0.2835,-0.002724,Long,Closed,45
46,46,Open,1.0,2021-04-01 13:30:00+00:00,106.8875,0.0,2021-05-10 13:30:00+00:00,117.0830,0.0,10.1955,0.095385,Long,Closed,46
47,47,Open,1.0,2021-05-21 13:30:00+00:00,117.2550,0.0,2021-08-19 13:30:00+00:00,136.9135,0.0,19.6585,0.167656,Long,Closed,47
48,48,Open,1.0,2021-08-20 13:30:00+00:00,138.4370,0.0,2021-08-23 13:30:00+00:00,141.0995,0.0,2.6625,0.019233,Long,Closed,48
49,49,Open,1.0,2021-08-24 13:30:00+00:00,142.3985,0.0,2021-09-10 13:30:00+00:00,141.9210,0.0,-0.4775,-0.003353,Long,Closed,49
50,50,Open,1.0,2021-09-13 13:30:00+00:00,143.4650,0.0,2021-09-14 13:30:00+00:00,143.4060,0.0,-0.0590,-0.000411,Long,Closed,50


In [ ]:
from savetopdf import save_backtesting_results_to_pdf
file_path = config['Data_filename_day'].split('.')[0]
file_path= (f"{file_path}_{config['Days']}")
save_backtesting_results_to_pdf(pf, file_path)